 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [453]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
#import scipy.sparse as sp
#from sklearn.datasets import load_iris


##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number



Pandas version: 1.4.3
Numpy version: 1.23.1


In [454]:
#Global variables
import_mode = True # import mode (True/False) - Imports an csv from PID
verbose_mode = False # verbose mode (True/False)
debug_mode = True # debug mode (True/False)
random_mode = False # random data generation  (True/False)
simulation_mode = False # simulation (True/False)

In [455]:
#Generate standard templated
generate_tag_number_definitions = False #Every element has a description and a unique number for software modules
#generate_element_list_alarm_matrix = False #Every element has a unique number of alarms
#generate_element_list_hardware_matrix = False #All possible combinations of elements have a unique IO

In [456]:
#Data generation
generate_element_list = False
generate_functional_matrix = False
generate_alarm_matrix = False
generate_hardware_config = False
generate_symbolic   = False
generate_s_plant    = False
generate_s_line = False
generate_flow_chart = False #--> GUI with active modes and phases, start condition [active modes list ] as bool (true false). Generarte excel which generete flow chart.


In [457]:
if import_mode: 
    ##Change working directory to temp/epic
    os.chdir('C://temp/epic/')
    cwd = os.getcwd()
    print('Current Working Directory is: ', cwd)
    


Current Working Directory is:  C:\temp\epic


In [458]:
if import_mode:
    print(os.listdir(cwd)) #List directory

['AutomationCode.xlsx', 'Info.txt', 'OLD_PID.csv', 'PID.xlsx', 'TND.xlsx', '~$AutomationCode.xlsx']


#Random_mode:
Generate aleatories PID numbers and symbols.



In [459]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [460]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

Series and DataFrames

In [461]:
#pid_series = pd.Series(random_PID_symbols, random_PID_numbers)
#pid_series = pid_series.sort_index(axis=0,ascending=True)
#pid_series

#Import Tag number definitions as a pandas series
#tnd = pd.read_csv('TND.csv',names=['PID','Description'])
#tnd = pd.read_excel('TND.xlsx',)
#tnd

In [462]:
#PID_series = tnd['PID'].squeeze()
#pid_exist = PID_series.str.contains('.100')

In [463]:
#tnd.at[0,'Description']

In [464]:
#tnd_pid = pd.read_excel('TND.xlsx',index_col='PID')
tnd_pid = pd.read_excel('TND.xlsx')
tnd_pid

,PID,Description
0,1.100,Intake air fan
1,1.101,Filter 1
2,1.102,Filter 2
3,1.103,Filter 3
4,1.104,NaN
...,...,...
895,1.995,NaN
896,1.996,NaN
897,1.997,NaN
898,1.998,NaN


In [465]:
#tnd_pid.at[1.100,'Description'] Works if index_col='PID'

Reading Excel File

We can read an excel file into a data frame:

pd.read_excel('myExcel.xlsx', index_col=['ColumnA'])

We can also load one sheet into a data frame:

pd.read_excel(open('myExcel.xlsx', 'rb'), sheet_name='Sheet1')

In [466]:
#df_data_excel = pd.read_excel('PID.xlsx', index_col='Prop_PID_Number')
df_data_excel = pd.read_excel('PID.xlsx')
print(df_data_excel)

     ShapeKey Prop_PID_Element  Prop_PID_Number
0       14434                M            1.100
1       16126               SC            1.100
2       14435                V            1.100
3       16125               XA            1.100
4       18990                F            1.101
..        ...              ...              ...
290     18102                P            2.304
291     18101             FSA-            2.308
292     18191                P            2.350
293     18192               XA            2.350
294     15757                L            2.990

[295 rows x 3 columns]


In [467]:
#Set PID first, second Element and ommit everything else.
df_data_excel = df_data_excel[['Prop_PID_Number','Prop_PID_Element']]
print(df_data_excel)

     Prop_PID_Number Prop_PID_Element
0              1.100                M
1              1.100               SC
2              1.100                V
3              1.100               XA
4              1.101                F
..               ...              ...
290            2.304                P
291            2.308             FSA-
292            2.350                P
293            2.350               XA
294            2.990                L

[295 rows x 2 columns]


In [468]:
#Not necessary anymore
#Remove ShapeKey from the dataframe
#del df_data_excel['ShapeKey']

In [469]:
df_data_excel.count()

Prop_PID_Number     295
Prop_PID_Element    295
dtype: int64

In [470]:
#Remove duplicates - WARNING: It removes duplicates alarms as 1.310 Particle counter 0.5 um and 5.0um (maybe add exception when is an PCounter and duplicates elements)

df_data_excel_uniques = df_data_excel.drop_duplicates()

In [471]:
df_data_excel_uniques.count()

Prop_PID_Number     287
Prop_PID_Element    287
dtype: int64

In [472]:
gkk = df_data_excel_uniques.groupby(['Prop_PID_Number', 'Prop_PID_Element'])
gkk.first()

Empty DataFrame
Columns: []
Index: [(1.1, M), (1.1, SC), (1.1, V), (1.1, XA), (1.101, F), (1.102, F), (1.108, PDI), (1.111, F), (1.112, F), (1.15, M), (1.15, SC), (1.15, TSA+), (1.15, V), (1.151, F), (1.152, F), (1.155, C), (1.158, PDI), (1.16, M), (1.16, SC), (1.16, V), (1.16, XA), (1.2, M), (1.2, SC), (1.2, V), (1.2, XA), (1.201, F), (1.208, PDI), (1.21, M), (1.21, SC), (1.21, V), (1.21, XA), (1.211, F), (1.218, PDI), (1.221, F), (1.232, F), (1.24, M), (1.24, SC), (1.24, V), (1.24, XA), (1.241, F), (1.242, F), (1.248, PDI), (1.3, FA-), (1.3, QIA+), (1.3, XA), (1.304, M), (1.304, P), (1.305, GSA+), (1.305, GSA-), (1.305, V), (1.305, Y), (1.308, FSA-), (1.31, FA-), (1.31, QIA+), (1.31, XA), (1.314, M), (1.314, P), (1.315, GSA+), (1.315, GSA-), (1.315, V), (1.315, Y), (1.318, FSA-), (1.32, FA-), (1.32, QIA+), (1.32, XA), (1.324, M), (1.324, P), (1.325, GSA+), (1.325, GSA-), (1.325, V), (1.325, Y), (1.328, FSA-), (1.33, FA-), (1.33, QIA+), (1.33, XA), (1.334, M), (1.334, P), (1.335, GSA+), (1.335, GSA-), (1.335, V), (1.335, Y), (1.338, FSA-), (1.35, P), (1.35, XA), (1.36, P), (1.36, XA), (1.37, P), (1.37, XA), (1.4, GIA+), (1.4, GIA-), (1.4, M), (1.4, SC), (1.4, XA), (1.41, GIA+), (1.41, GIA-), (1.41, M), (1.41, SC), (1.41, XA), (1.52, GSA+), (1.52, GSA-), ...]

[287 rows x 0 columns]

In [473]:
#Get uniques values from the PID Elements (the columns for the matrix)
df_unique_elememts = df_data_excel.Prop_PID_Element.unique()
df_unique_elememts.sort()
print(df_unique_elememts)




['B' 'C' 'D' 'EC' 'F' 'FA-' 'FIA+' 'FICA±' 'FSA-' 'GIA' 'GIA+' 'GIA-'
 'GSA+' 'GSA-' 'GZA-' 'H' 'K' 'L' 'LSA-' 'M' 'MIA±' 'NC' 'P' 'PDI'
 'PDICA±' 'PDIS' 'PI' 'PSA-' 'QIA+' 'QIA-' 'QSA+' 'S' 'SC' 'SIA±' 'T'
 'TIC' 'TICA±' 'TSA+' 'V' 'W' 'WICA±' 'XA' 'Y' 'YC']


#Export Unique elements to excel file - Once


In [474]:
#Create a dataframe containing the elements
#pd_automation_code = pd.DataFrame(df_unique_elememts)
#Create the writer object
#automation_code = pd.ExcelWriter('AutomationCode.xlsx')
#Write dataframe to excel file
#pd_automation_code.to_excel(automation_code)
#Save excel 
#automation_code.save()
#Close excel
#automation_code.close()


In [475]:
#Sort by PID number
df_data_excel_sorted = df_data_excel_uniques.sort_values(by='Prop_PID_Number')

In [476]:
#Set PID number as multi-index (group by does better)
#df_data_excel_sorted.set_index(['Prop_PID_Number','Prop_PID_Element'])

In [477]:
#df_data_excel_sorted.head(1000)

In [478]:
#Add column, load data from Automation Code and mix
df_data_excel_sorted['Automation Code'] = ''


In [479]:
df_data_excel_sorted

,Prop_PID_Number,Prop_PID_Element,Automation Code
0,1.100,M,
1,1.100,SC,
2,1.100,V,
3,1.100,XA,
4,1.101,F,
...,...,...,...
289,2.304,M,
291,2.308,FSA-,
293,2.350,XA,
292,2.350,P,


In [480]:
#Load Automation Code excel file
df_automation_code = pd.read_excel('AutomationCode.xlsx')
df_automation_code.head()

,Prop_PID_Element,Automation_Code
0,B,*Buffer *
1,C,*Catalizator *
2,D,* D *
3,EC,Voltage Control
4,F,*Filter*


In [481]:
#Lookup
df = df_data_excel
df['Automation_Code'] = ''
print (df)

     Prop_PID_Number Prop_PID_Element Automation_Code
0              1.100                M                
1              1.100               SC                
2              1.100                V                
3              1.100               XA                
4              1.101                F                
..               ...              ...             ...
290            2.304                P                
291            2.308             FSA-                
292            2.350                P                
293            2.350               XA                
294            2.990                L                

[295 rows x 3 columns]


In [482]:
# Deprecated df['Automation Code'] = df.set_index('Prop_PID_Element').lookup(df_automation_code.Prop_PID_Element, df_automation_code.Automation_Code)

In [483]:
#pd.melt(df,)
#print(df.combine_first(df_automation_code))

In [484]:
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [485]:
df['Automation_Code'] = df['Prop_PID_Element'].apply(xlookup, args = (df_automation_code['Prop_PID_Element'], df_automation_code['Automation_Code']));

In [486]:
df

,Prop_PID_Number,Prop_PID_Element,Automation_Code
0,1.100,M,* Motor *
1,1.100,SC,Speed Control
2,1.100,V,* Ventilator *
3,1.100,XA,Alarm
4,1.101,F,*Filter*
...,...,...,...
290,2.304,P,* Pump *
291,2.308,FSA-,Flow Switching Alarm -
292,2.350,P,* Pump *
293,2.350,XA,Alarm


In [487]:
print(df.to_string())

     Prop_PID_Number Prop_PID_Element                                 Automation_Code
0              1.100                M                                       * Motor *
1              1.100               SC                                   Speed Control
2              1.100                V                                  * Ventilator *
3              1.100               XA                                           Alarm
4              1.101                F                                        *Filter*
5              1.102                F                                        *Filter*
6              1.108              PDI                  Pressure Difference Indication
7              1.111                F                                        *Filter*
8              1.112                F                                        *Filter*
9              1.150                M                                       * Motor *
10             1.150               SC                 

In [488]:
##Add Tag number definition to df
print(tnd_pid)


       PID     Description
0    1.100  Intake air fan
1    1.101        Filter 1
2    1.102        Filter 2
3    1.103        Filter 3
4    1.104             NaN
..     ...             ...
895  1.995             NaN
896  1.996             NaN
897  1.997             NaN
898  1.998             NaN
899  1.999             NaN

[900 rows x 2 columns]


In [489]:
df['Tag_Number_Definition'] = df['Prop_PID_Number'].apply(xlookup, args = (tnd_pid['PID'], tnd_pid['Description']));

In [490]:
print(df.to_string())

     Prop_PID_Number Prop_PID_Element                                 Automation_Code                 Tag_Number_Definition
0              1.100                M                                       * Motor *                        Intake air fan
1              1.100               SC                                   Speed Control                        Intake air fan
2              1.100                V                                  * Ventilator *                        Intake air fan
3              1.100               XA                                           Alarm                        Intake air fan
4              1.101                F                                        *Filter*                              Filter 1
5              1.102                F                                        *Filter*                              Filter 2
6              1.108              PDI                  Pressure Difference Indication           Diferential pressure sensor
7       